In [245]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError, ReadTimeout
import pandas as pd
from time import sleep

In [ ]:
def fetchAndSaveToFile(url, path, retries=3, backoff_factor=1):
    headers = {'User-Agent': 'Mozilla/5.0'}
    for attempt in range(retries):
        try:
            # Increased timeout from 10 to 20 seconds
            r = requests.get(url, headers=headers, timeout=20)
            with open(path, "w", encoding='utf-8') as f:
                f.write(r.text)
            break  # Success, exit the loop
        except (ConnectionError, ReadTimeout) as e:
            print(f"Attempt {attempt + 1} failed with error: {e}")
            sleep(backoff_factor * (2 ** attempt))  # Exponential backoff
    else:  # All attempts failed
        print(f"Failed to fetch data after {retries} attempts.")


def beauty(path):
    try:
        with open(path, "r", encoding='utf-8') as f:
            html_doc = f.read()
            soup = BeautifulSoup(html_doc, 'html.parser')
            new_content = soup.prettify()
        with open("new_content.html", "w", encoding='utf-8') as f:
            f.write(new_content)
    except Exception as e:
        print(f"Error processing file: {e}")

price_list = []; emi_list = []; img_list = []; appartment_name_list = []; appartment_by_list = []; size_list = []; details_list=[]; desc_list = []; places_list = []


for p in range(1, 173):
    url = "https://housing.com/buy-apartments-in-jaipur-for-sale-srpid-M1P268dw8h2rtw7ykpy?page=" + str(p)

    fetchAndSaveToFile(url, "content.html")
    beauty("content.html")
    with open(R"D:\Housing_DSc\Housing_Data_Science\housing_data_science\notebooks\new_content.html","r", encoding="utf-8") as file:
        html_doc = file.read()
        soup = BeautifulSoup(html_doc, 'html.parser')

    # print(soup.title)
    article_tag = soup.findAll('article', attrs={'data-testid': 'card-container'})
    # print(script_tag)

    for k in range(0,len(article_tag)):

        try:
            price = article_tag[k].findAll('div', attrs={'data-testid': 'priceid'})
            price_list.append(price[0].text.strip())
        except:
            price_list.append(None)

        
        try:
            emi = article_tag[k].findAll('span', attrs={'class': '_9jtlke _l881bl _gzftgi _7l1994 _c81fwx _g3dlk8 T_9671ecd9'})
            emi_list.append(emi[0].text.strip())
        except:
            emi_list.append(None)

        try:
            img = article_tag[k].find('div', attrs={'class': 'css-uwwqev'})
            img_list.append(img.img['src'])
        except Exception as e:
            img_list.append(None)

        
        try:
            appartment_name = article_tag[k].findAll('div', attrs={'class': '_9s1txw _gqyh40 _0h1q9y T_8c8ed98f'})
            appartment_name_list.append(appartment_name[0].text.strip().split("\n")[0])
        except:
            appartment_name_list.append(None)

        
        try:
            appartment_by = article_tag[k].findAll('div', attrs={'class': '_c81fwx _g3dlk8 _7l1v10 T_1bbfaea9'})
            appartment_by_list.append(appartment_by[0].text.strip().split("\n")[0])
        except:
            appartment_by_list.append(None)

        
        try:
            size = article_tag[k].findAll('h3', attrs={'class': 'T_091c165f _sq1l2s _vv1q9c _ks15vq T_79c363f6 _5vy24jg8 _blas14la _csbfng _g3dlk8 _c81fwx _h3ftgi'})
            size_list.append(" ".join(size[0].text.strip().split()))
        except:
            size_list.append(None)

        
        try:
            details = article_tag[k].findAll('section', attrs={'class': 'T_ea8ae345 _1enfn7od _12el1ule _12il1osq _11ar1l2s _1vd01q9c _orqr15vq _cbben7od _3nng1e54 _12eccj1k _dgd0cs5v _gcpm15vq _9j73ad _gnftgi _9s1txw'})
            details_list.append(" ".join(details[0].text.strip().split()))
        except:
            details_list.append(None)

        
        try:
            desc = article_tag[k].findAll('div', attrs={'data-q': 'desc'})
            desc_list.append(" ".join(desc[0].text.split()[:-1]))
        except:
            desc_list.append(None)

        
        try:
            places = article_tag[k].findAll('div', attrs={'class': 'T_aa486ccc _be1g80 _n5ftgi _0h1h6o _9s1txw _h3exct _84ftgi'})
            places_list.append(" ".join(places[0].text.split()[:-1]))
        except:
            places_list.append(None)
    print("Page ->",p)

In [249]:
dic = {
    "Price": price_list,
    "EMI": emi_list,
    "Image": img_list,
    "Appartment Name": appartment_name_list,
    "Appartment By": appartment_by_list,
    "Size": size_list,
    "Details": details_list,
    "Description": desc_list,
    "Places": places_list
}

In [251]:
pd.DataFrame(dic).to_csv("housing_data.csv", index=False)